# Análises - NYC Taxi Data (Camada de Consumo)

Este notebook contém as análises solicitadas no case técnico iFood.
Todas as queries utilizam a camada **Trusted** como fonte de dados.

**Autor:** Vinicius  
**Data:** 2025

## Perguntas a serem respondidas:
1. Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?
2. Qual a média de passageiros (passenger_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?

## 1. Média de Valor Total por Mês

Esta análise calcula a média do valor total (total_amount) recebido mensalmente,
considerando todas as corridas de yellow táxis registradas no período.

**Lógica:**
- Extrai o ano e mês da data de pickup
- Agrupa por período (ano-mês)
- Calcula a média do total_amount
- Arredonda para 2 casas decimais
- Ordena cronologicamente

**Filtros aplicados:**
- `total_amount > 0`: Exclui valores negativos ou zero (possíveis estornos/cancelamentos)
- `total_amount IS NOT NULL`: Garante apenas dados válidos

In [0]:
SELECT 
    YEAR(tpep_pickup_datetime) AS ano,
    MONTH(tpep_pickup_datetime) AS mes,
    CONCAT(
        YEAR(tpep_pickup_datetime), 
        '-', 
        LPAD(MONTH(tpep_pickup_datetime), 2, '0')
    ) AS periodo,
    ROUND(AVG(total_amount), 2) AS media_valor_total,
    COUNT(*) AS total_corridas,
    ROUND(SUM(total_amount), 2) AS valor_total_periodo,
    ROUND(MIN(total_amount), 2) AS valor_minimo,
    ROUND(MAX(total_amount), 2) AS valor_maximo
FROM ifood_catalog.trusted_layer.tb_taxi_data_for_analysis
WHERE total_amount IS NOT NULL
    AND total_amount > 0
    AND YEAR(tpep_pickup_datetime) = 2023
    AND MONTH(tpep_pickup_datetime) <= 5
GROUP BY 
    YEAR(tpep_pickup_datetime),
    MONTH(tpep_pickup_datetime)
ORDER BY ano, mes;

## 2. Média de Passageiros por Hora - Maio/2023

Esta análise calcula a média de passageiros por hora do dia especificamente
para o mês de maio de 2023, considerando todas as corridas da frota.

**Lógica:**
- Filtra apenas corridas de maio/2023
- Extrai a hora do pickup_datetime (0-23)
- Agrupa por hora
- Calcula média de passenger_count
- Arredonda para 2 casas decimais
- Inclui total de corridas por hora para contexto

**Filtros aplicados:**
- `YEAR = 2023 AND MONTH = 5`: Apenas maio/2023
- `passenger_count > 0`: Exclui corridas sem passageiros registrados
- `passenger_count IS NOT NULL`: Garante dados válidos

In [0]:
SELECT 
    HOUR(tpep_pickup_datetime) AS hora_do_dia,
    ROUND(AVG(passenger_count), 2) AS media_passageiros,
    COUNT(*) AS total_corridas,
    ROUND(SUM(passenger_count), 0) AS total_passageiros,
    ROUND(MIN(passenger_count), 0) AS min_passageiros,
    ROUND(MAX(passenger_count), 0) AS max_passageiros
FROM ifood_catalog.trusted_layer.tb_taxi_data_for_analysis
WHERE YEAR(tpep_pickup_datetime) = 2023
    AND MONTH(tpep_pickup_datetime) = 5
    AND passenger_count IS NOT NULL
    AND passenger_count > 0
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY hora_do_dia;